EVALUACIÓN MÓDULO 3 : PROBLEMA 1

En el archivo auto.csv se encuentran los siguientes datos de diferentes automóviles:
Cilindros,Cilindrada,Potencia,Peso,Aceleración,Año del coche,Origen y Consumo (mpg). Las unidades de las características de los automóviles no se encuentran en el sistema internacional. La variable “origen” es un código que identifica al país de origen.
Crea un modelo con él para que se pueda estimar el consumo de un vehículo a partir del resto de las variables.

In [30]:
%pylab
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['var']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Importamos el fichero de datos usando Pandas

In [31]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

auto = pd.read_csv('auto.csv', sep = ',')
auto.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,mpg
0,8,307.0,130.0,3504.0,12.0,70,1,18.0
1,8,350.0,165.0,3693.0,11.5,70,1,15.0
2,8,318.0,150.0,3436.0,11.0,70,1,18.0
3,8,304.0,150.0,3433.0,12.0,70,1,16.0
4,8,302.0,140.0,3449.0,10.5,70,1,17.0


De un primer vistazo al dataframe con los datos , vemos que hay dos variables que podrían considerarse categóricas , el número de cilindros y el año del modelo , por lo que buscamos los valores tomados por dichas variables:

In [32]:
auto['cylinders'].unique()


array([8, 4, 6, 3, 5])

In [33]:
auto['model_year'].unique()

array([70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82])

Puesto que la variable "año del modelo" toma 13 valores distintos, parece demasiado grande el número de valores como para considerarla categórica; no así ocurre con la variable "número de cilindros" que solo toma 5 valores distintos y,además,si se conoce que intentamos modelizar el consumo de un coche parece buena idea segmentar los datos por número de cilindros, por lo que vamos a crear variables dummy para cada valor de dicha variable mediante el método que nos proporciona Pandas y las añadimos al dataframe de los datos existente 

In [34]:
auto = pd.concat([auto, pd.get_dummies(auto['cylinders'])], axis = 1)
auto.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,mpg,3,4,5,6,8
0,8,307.0,130.0,3504.0,12.0,70,1,18.0,0,0,0,0,1
1,8,350.0,165.0,3693.0,11.5,70,1,15.0,0,0,0,0,1
2,8,318.0,150.0,3436.0,11.0,70,1,18.0,0,0,0,0,1
3,8,304.0,150.0,3433.0,12.0,70,1,16.0,0,0,0,0,1
4,8,302.0,140.0,3449.0,10.5,70,1,17.0,0,0,0,0,1


Puesto que en el enunciado del problema se nos avisa de que las unidades de las características de los automóviles no se encuentran en el sistema internacional , procedemos a normalizar los datos ,asumiendo que las distribuciones de los valores que toman las características son homogéneas. La normalización se lleva a cabo restando el valor mínimo a cada uno de los valores y se divide por la diferencia entre el valor máximo y el mínimo. Se utiliza la siguiente expresión para normalizar la característica:
                         x^mm = (x-l)/(h-l) , 
donde x es la característica original, l el valor mínimo, h el máximo y x^mm es el resultado de la característica normalizada. De esta forma se consiguen nuevas características en las que los datos se encuentran distribuidos de la misma forma que los originales, pero con valores entre 0 y 1 e independientes de las unidades.
Lo hacemos con el objeto MinMaxScaler:

In [35]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(auto)
autoN=pd.DataFrame(scaler.transform(auto))
autoN.columns=auto.columns
autoN.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,mpg,3,4,5,6,8
0,1.0,0.617571,0.456522,0.536150,0.238095,0.0,0.0,0.239362,0.0,0.0,0.0,0.0,1.0
1,1.0,0.728682,0.646739,0.589736,0.208333,0.0,0.0,0.159574,0.0,0.0,0.0,0.0,1.0
2,1.0,0.645995,0.565217,0.516870,0.178571,0.0,0.0,0.239362,0.0,0.0,0.0,0.0,1.0
3,1.0,0.609819,0.565217,0.516019,0.238095,0.0,0.0,0.186170,0.0,0.0,0.0,0.0,1.0
4,1.0,0.604651,0.510870,0.520556,0.148810,0.0,0.0,0.212766,0.0,0.0,0.0,0.0,1.0


Separamos la variable objetivo de las explicativas

In [36]:
target = 'mpg'
features = list(autoN.columns)
features.remove('mpg')

# Listado de variables disponibles para hacer un modelo.
for var in features:
    print(var , ':' , len(set(autoN[var])))




cylinders : 5
displacement : 81
horsepower : 93
weight : 346
acceleration : 95
model_year : 13
origin : 3
3 : 2
4 : 2
5 : 2
6 : 2
8 : 2


Puesto que en el enunciado del problema se nos avisa de que la variable Origen es un identificador , procedemos a eliminarla,así como la variable "número de cilindros" que ya está incorporada al dataframe mediante las variables dummy recientemente añadidas.


In [37]:
features.remove('origin')
features.remove('cylinders')
autoN[features].head()

,displacement,horsepower,weight,acceleration,model_year,3,4,5,6,8
0,0.617571,0.456522,0.536150,0.238095,0.0,0.0,0.0,0.0,0.0,1.0
1,0.728682,0.646739,0.589736,0.208333,0.0,0.0,0.0,0.0,0.0,1.0
2,0.645995,0.565217,0.516870,0.178571,0.0,0.0,0.0,0.0,0.0,1.0
3,0.609819,0.565217,0.516019,0.238095,0.0,0.0,0.0,0.0,0.0,1.0
4,0.604651,0.510870,0.520556,0.148810,0.0,0.0,0.0,0.0,0.0,1.0


Creación de un modelo
Una vez transformadas las variables se puede proceder a la creación de un modelo.

In [38]:
use_features = features[:]
data_model = autoN[use_features]


Procedemos a averiguar si existe colinealidad entre variables explicativas

In [39]:
from sklearn.linear_model import LinearRegression

def calculateVIF(data):
    
    
    model = LinearRegression()
    features = list(data.columns)
    num_features = len(features)
    result = pd.DataFrame(index = ['VIF'], columns = features)
    result = result.fillna(0)
    
    for ite in range(num_features):
        x_features = features[:]
        y_featue = features[ite]
        x_features.remove(y_featue)
        
        x = data[x_features]
        y = data[y_featue]
        
        model.fit(data[x_features], data[y_featue])
        
        result[y_featue] = 1/(1 - model.score(data[x_features], data[y_featue]))
    
    return result

def selectDataUsingVIF(data, max_VIF = 5):
    result = data.copy(deep = True)
    
    VIF = calculateVIF(result)
    
    while VIF.as_matrix().max() > max_VIF:
        col_max = np.where(VIF == VIF.as_matrix().max())[1][0]
        features = list(result.columns)
        features.remove(features[col_max])
        result = result[features]
        
        VIF = calculateVIF(result)
        
    return result

In [40]:
calculateVIF(data_model)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in double_scalars


,displacement,horsepower,weight,acceleration,model_year,3,4,5,6,8
VIF,20.083967,10.186015,11.289679,2.682099,1.251307,inf,inf,inf,inf,inf


Al revisar el listado con los resultados del VIF, se observa que hay una característica con VIF muy elevado (cilindrada) , tres más con VIF elevado (Cilindros,Potencia y Peso) así como todas las dummy asociadas al número de cilindros que tienen valor infinito,  lógico por otra parte puesto que a mayor número de cilindros , mayor cilindrada ('displacement') .Una vez obtenido el VIF, hay que definir un proceso para la eliminación de características. 

In [41]:
model_vars=selectDataUsingVIF(data_model)
calculateVIF(model_vars)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


,weight,acceleration,model_year,5,6
VIF,1.337895,1.328668,1.149811,1.022776,1.073296


Como vemos , se han eliminado varias variables que provocaban multicolinealidad;a partir de esto separamos los datos en conjunto de muestra y validación .

In [42]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(model_vars, autoN[target])

Creamos el modelo para predecir el consumo a partir de los datos proporcionados:

In [43]:
model = LinearRegression()
model.fit(x_train, y_train)

predit_train = model.predict(x_train)
predit_test = model.predict(x_test)

# Evaluación de R2
print('R2 en entrenamiento es: ', model.score(x_train, y_train))
print('R2 en validación es: ', model.score(x_test, y_test))

R2 en entrenamiento es:  0.8183477026499946
R2 en validación es:  0.8515178468188918


Como vemos,los coeficientes R2 son similares en entrenamiento como en validación , por lo que podemos suponer que no hay sobreajuste en el modelo seleccionado .Obtenemos también los coeficientes del modelo y los valores de los errores y vemos que estos son bastante bajos , por lo que podemos considerar el modelo obtenido un buen modelo para estimar el consumo en función de las características proporcionadas en el fichero de datos

In [44]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error

print('Coefficients: \n', model.coef_)
print('Independent term: \n', model.intercept_)
print('Error cuadrático medio', mean_squared_error(predit_test, y_test))
print('Error absoluto medio', mean_absolute_error(predit_test, y_test))
print('Mediana del error absoluto', median_absolute_error(predit_test, y_test))

Coefficients: 
 [-0.59985677  0.0408598   0.24094253  0.04222566 -0.06623996]
Independent term: 
 0.4931503485798366
Error cuadrático medio 0.006759731427838652
Error absoluto medio 0.06118076261562979
Mediana del error absoluto 0.041865940263549686
